In [ ]:
#| default_exp segment_seismic

In [ ]:
#| export
import torch
import import_ipynb
from argparse import ArgumentParser
from datetime import datetime
from tqdm import tqdm
from math import ceil
from sklearn.model_selection import KFold
import os

import libs
from libs.loader.data_loader import *
from libs.models.load_empty_model import load_empty_model
from libs.metrics import RunningMetrics, EarlyStopper
from libs.utils import store_results
from libs.loss.cross_entropy import CrossEntropyLoss
setattr(libs.loss, 'CrossEntropyLoss', CrossEntropyLoss)

# Segmentation Benchmark

## Command Line Arguments

### Overview
This script `segment_seismic.py` allows training and testing of seismic image segmentation models. It provides various command-line arguments to customize the behavior of the script, including the choice of model architecture, data paths, hyperparameters, and more.

### Usage Examples
To train a model from scratch, use the following command:
```bash
python3 segment_seismic.py -t -a unet -d path/to/data.npy -l path/to/labels.npy
```
To test a previously trained model, omit the `-t` flag and provide the path to the stored model using the `-m` option:
```bash
python3 segment_seismic.py -a unet -d path/to/data.npy -l path/to/labels.npy -m path/to/model.pt
```

### Command Line Arguments

| Argument                | Description                                                                                                                                                                                     | Default    |
|-------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------|
| `-a`, `--architecture`  | Choose a model architecture for training. Options: `segnet`, `unet`, `deconvnet`.                                                                                                              |            |
| `-d`, `--data-path`     | Path to the seismic volume file in `.npy` or `.segy` format.                                                                                                                                    |            |
| `-l`, `--labels-path`   | Path to the labels file in `.npy` format.                                                                                                                                                       |            |
| `-t`, `--train`         | Train a model from scratch. If `False`, requires a path to a stored model (`-m`).                                                                                                               | `False`    |
| `-b`, `--batch-size`    | Size of the training batch.                                                                                                                                                                     | `16`       |
| `-D`, `--device`        | Choose GPU device for training. Defaults to CPU if GPU isn't available.                                                                                                                         | `cuda:0`   |
| `-v`, `--cross-validation` | Enable 5-Fold Cross Validation during training.                                                                                                                                              | `False`    |
| `-L`, `--loss-function`  | Loss function to use. Options: `cel` (Cross Entropy Loss).                                                                                                                                     | `cel`      |
| `-o`, `--optimizer`     | Optimizer to use. Options: `adam`, `sgd` (Stochastic Gradient Descent).                                                                                                                        | `adam`     |
| `-r`, `--learning-rate` | Learning rate for training.                                                                                                                                                                     | `1e-4`     |
| `-w`, `--weight-decay`  | L2 regularization weight decay. `0` indicates no decay.                                                                                                                                        | `1e-5`     |
| `-W`, `--weighted-loss` | Use class weights in the loss function to handle class imbalance.                                                                                                                               | `False`    |
| `-e`, `--n-epochs`      | Number of training epochs.                                                                                                                                                                      | `20`       |
| `-O`, `--orientation`    | Choose orientation for slices: `in` (inlines) or `cross` (crosslines).                                                                                                                          | `in`       |
| `-f`, `--faulty-slices-list` | Path to a `.json` file listing indexes of faulty slices to exclude from training.                                                                                                             | `None`     |
| `-F`, `--few-shot`      | Swap train and test sets to train on less data.                                                                                                                                                 | `False`    |
| `-m`, `--model-path`    | Path to a stored model file (`.pt`) for testing. Ignored if training (`-t`) is enabled.                                                                                                       | `None`     |
| `-p`, `--results-path`  | Directory for storing results. Metrics will be saved in a `.json` file, and predictions in `.png` format will be saved in a folder. If training, model weights will be saved as `.pt` files. | `results`  |

### Notes
- The `-a`, `-d`, and `-l` options are required for both training and testing.
- When testing (`-t` is `False`), provide the path to a stored model (`-m`) to load the model weights.
- For training, additional arguments like `-b`, `-D`, `-v`, `-L`, `-o`, `-r`, `-w`, `-W`, `-e`, `-O`, `-f`, `-F`, and `-p` can be used to configure the training process and output directory.
- Use the `-v` option to enable 5-fold cross-validation during training.
- If class imbalance is present, consider using `-W` to enable weighted loss.
- `-F` can be used to train on a smaller subset of data by swapping the train and test sets.
- The `--faulty-slices-list` option allows excluding faulty slices from the training data.
- Results, including metrics and predictions, will be stored in the specified `results` directory.
- In order to execute the benchmark in notebook format, you need to manually configure the arguments within the main cell

## `train_test_split`

### **Description:**
- Splits the dataset into train and test sets either using cross-validation or a simple train-test split.

### **Parameters:**

- `args` (object): An object containing arguments and configurations for the splitting process.
- `dataset` (list or numpy array): The dataset to be split.

### **Returns:**
- `list`: A list of tuples where each tuple contains indices for the train and test sets.

## `train`

### **Description:**
- Trains a neural network model using the provided dataset, configurations, and loss criterion.

### **Parameters:**

- `args` (object): An object containing training arguments and configurations.
- `dataset` (torch Dataset): The dataset used for training.
- `device` (torch device): The device on which to perform training (e.g., 'cpu', 'cuda').
- `criterion` (torch.nn.Module): The loss criterion used for optimization.
- `n_classes` (int): The number of classes in the dataset.
- `indices` (list): List of indices indicating which examples to use for training.

### **Returns:**
- `model` (torch.nn.Module): The trained model.
- `results` (dict): A dictionary containing training results including train scores and losses.

## `test`

### **Description:**
- Tests a trained neural network model using the provided dataset, configurations, and loss criterion.

### **Parameters:**

- `args` (object): An object containing testing arguments and configurations.
- `dataset` (torch Dataset): The dataset used for testing.
- `device` (torch device): The device on which to perform testing (e.g., 'cpu', 'cuda').
- `criterion` (torch.nn.Module): The loss criterion used for evaluation.
- `n_classes` (int): The number of classes in the dataset.
- `indices` (list): List of indices indicating which examples to use for testing.
- `fold` (int): The fold number used for testing in cross-validation.
- `model` (torch.nn.Module, optional): The trained model to be tested. If None, the model is loaded from the specified model path.

### **Returns:**
- `preds` (dict): A dictionary containing the predicted outputs for each test example.
- `results` (dict): A dictionary containing testing results including test scores and losses.

## `run`

### **Description:**
- Executes the training and testing pipeline based on the provided arguments and configurations.

### **Parameters:**

- `args` (object): An object containing all the necessary arguments and configurations for the training and testing pipeline.

### **Returns:**
- None

## `if __name__ == '__main__':`

### **Description:**
- This block of code serves as the entry point for the script when executed directly. It defines and parses command-line arguments using the `ArgumentParser` from the `argparse` module, then calls the `run` function with the parsed arguments.

### **Parameters:**
- None

### **Usage:**
- This block of code should be placed at the end of a script file to allow for direct execution of the script with command-line arguments.

In [ ]:
#| export
def train_test_split(args, dataset):
    if args.cross_validation:
        kf = KFold(n_splits=5, shuffle=False)

        if args.few_shot:
            # 20% train / 80% test
            splits = [
                (test_idx.tolist(), train_idx.tolist()) for train_idx, test_idx in kf.split(dataset)
            ]
        else:
            # 80% train / 20% test
            splits = [
                (train_idx.tolist(), test_idx.tolist()) for train_idx, test_idx in kf.split(dataset)
            ]
    else:
        if args.few_shot:
            # 20% train / 80% test
            test_ratio = 0.8
        else:
            # 80% train / 20% test
            test_ratio = 0.2

        test_size  = int(len(dataset) * test_ratio)
        splits = [(list(range(test_size, len(dataset))), list(range(0, test_size)))]

    return splits

In [ ]:
#| export
def train(args, dataset, device, criterion, n_classes, indices):
    print('\nCreating model...')
    print('Architecture:   ', args.architecture.upper())
    print('Optimizer:      ', args.optimizer)
    print('Device:         ', device)
    print('Loss function:  ', args.loss_function)
    print('Learning rate:  ', args.learning_rate)
    print('Batch size:     ', args.batch_size)
    print('N. of epochs:   ', args.n_epochs)

    print('\nNumber of train examples:', len(indices))

    print('\nWeighted loss ENABLED' if args.weighted_loss else 'Weighted loss DISABLED')
    print(f'Training with {"INLINES" if args.orientation == "in" else "CROSSLINES"}')

    train_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=args.batch_size,
        shuffle=True
    )

    model = load_empty_model(args.architecture, n_classes)
    model = model.to(device)

    # Defining the optimizer
    optimizer_map = {
        'adam': torch.optim.Adam,
        'sgd' : torch.optim.SGD
    }

    OptimizerClass = optimizer_map[args.optimizer]
    optimizer = OptimizerClass(
        params=model.parameters(),
        lr=args.learning_rate,
        weight_decay=args.weight_decay
    )

    # Initializing metrics
    train_metrics = RunningMetrics(n_classes=n_classes, bf1_threshold=2)

    # Storing the loss for each epoch
    train_loss_list = []

    for epoch in range(args.n_epochs):
        # Training phase
        model.train()
        train_loss = 0

        print(datetime.now().strftime('\n%Y/%m/%d %H:%M:%S'))
        print(f'Training on epoch {epoch + 1}/{args.n_epochs}\n')

        for images, labels in tqdm(train_loader, ascii=' >='):
            optimizer.zero_grad()

            images = images.type(torch.FloatTensor).to(device)
            labels = labels.type(torch.FloatTensor).to(device)

            outputs = model(images)

            # Updating the running metrics
            train_metrics.update(images=outputs, targets=labels)

            # Computing the loss and updating weights
            loss = criterion(images=outputs, targets=labels.long())
            train_loss += loss.item()
            loss.backward()

            optimizer.step()
        
        train_loss = train_loss / ceil((len(train_loader) / args.batch_size))
        train_scores = train_metrics.get_scores()

        print(f'Train loss: {train_loss}')
        print(f'Train mIoU: {train_scores["mean_iou"]}')

        train_loss_list.append(train_loss)
        train_metrics.reset()

    results = {
        'train_scores' : train_scores,
        'train_losses' : train_loss_list,
    }
    
    return model, results

In [ ]:
#| export
def test(args, dataset, device, criterion, n_classes, indices, fold, model=None):
    print('\nTesting model...')
    print('Architecture:   ', args.architecture.upper())
    print('Device:         ', device)
    print('Loss function:  ', args.loss_function)
    print('Batch size:     ', args.batch_size)

    print(f'\nNumber of test examples: {len(indices)} (slices {indices[0]} to {indices[-1]})')

    print('\nWeighted loss ENABLED' if args.weighted_loss else 'Weighted loss DISABLED')
    print(f'Testing with {"INLINES" if args.orientation == "in" else "CROSSLINES"}')
    
    test_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=args.batch_size,
        shuffle=False
    )

    # Initializing metrics
    test_metrics  = RunningMetrics(n_classes=n_classes, bf1_threshold=2)

    # Storing the loss for each epoch
    test_loss_list  = []

    if model is None:
        print(f'\nTraining is OFF. Loading stored model from {args.model_path}')

        model = load_empty_model(args.architecture, n_classes)
        model = model.to(device)
        
        # model_path = os.path.join(args.model_path, f'model_fold_{fold}.pt')

        if not os.path.isfile(args.model_path):
            raise FileNotFoundError(f'No such file or directory for stored model: {args.model_path}')

        # Loading previously stored model
        model.load_state_dict(torch.load(args.model_path))
    
    preds = {}
    slice_idx = indices[0]

    # Testing phase
    with torch.no_grad():
        model.eval()
        test_loss = 0

        print(datetime.now().strftime('\n%Y/%m/%d %H:%M:%S'))
        print('Testing the model...\n')

        for images, labels in tqdm(test_loader, ascii=' >='):
            images = images.type(torch.FloatTensor).to(device)
            labels = labels.type(torch.FloatTensor).to(device)

            outputs = model(images)

            # Iterating over the batch
            for pred in outputs:
                preds[slice_idx] = pred
                slice_idx += 1

            # Updating the running metrics
            test_metrics.update(images=outputs, targets=labels)

            # Computing the loss
            loss = criterion(images=outputs, targets=labels.long())
            test_loss += loss.item()
        
        test_loss = test_loss / ceil((len(test_loader) / args.batch_size))
        test_scores = test_metrics.get_scores()

        print(f'Test loss: {test_loss}')
        print(f'Test mIoU: {test_scores["mean_iou"]}')

        test_loss_list.append(test_loss)
        test_metrics.reset()
    
    results = {
        'test_scores'  : test_scores,
        'test_losses'  : test_loss_list,
    }

    return preds, results

In [ ]:
#| export
def run(args):
    print('')
    print('Data path:    ', args.data_path)
    print('Labels path:  ', args.labels_path)
    print('Results path: ', args.results_path)
    print('')

    print('Loading dataset...')

    dataset = SeismicDataset(
        data_path=args.data_path,
        labels_path=args.labels_path,
        orientation=args.orientation,
        compute_weights=args.weighted_loss,
        faulty_slices_list=args.faulty_slices_list
    )

    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')

    if args.weighted_loss:
        class_weights = torch.tensor(dataset.get_class_weights(), device=device, requires_grad=False)
        class_weights = class_weights.float()
    else:
        class_weights = None

    loss_map = {
        'cel': ('CrossEntropyLoss', {'reduction': 'sum', 'weight': class_weights})
    }

    # Defining the loss function
    loss_name, loss_args = loss_map[args.loss_function]
    criterion = getattr(libs.loss, loss_name)(**loss_args)

    # Splitting the data into train and test
    splits = train_test_split(args, dataset)

    results = []

    for fold_number, (train_indices, test_indices) in enumerate(splits):
        if args.cross_validation:
            print(f'\n======== FOLD {fold_number + 1}/5 ========')
        
        train_indices, test_indices = splits[fold_number]
        
        train_set = torch.utils.data.Subset(dataset, train_indices)
        test_set = torch.utils.data.Subset(dataset, test_indices)

        if args.train:
            model, train_results = train(
                args,
                train_set,
                device=device,
                criterion=criterion,
                n_classes=dataset.get_n_classes(),
                indices=train_indices
            )
        else:
            model = None
            train_results = {}
            # train_results = load_train_results()

        preds, test_results = test(
            args,
            test_set,
            device=device,
            criterion=criterion,
            n_classes=dataset.get_n_classes(),
            indices=test_indices,
            fold=fold_number,
            model=model
        )

        results.append({
            'model': model,
            'preds': preds,
            'train_indices': train_indices,
            'test_indices': test_indices,
            **train_results,
            **test_results
        })

    store_results(args, results, n_classes=dataset.get_n_classes())

In [ ]:
#| export
if __name__ == '__main__':
    parser = ArgumentParser(description='Hyperparameters')
    print(parser)
    parser.add_argument('-a', '--architecture',
        dest='architecture',
        type=str,
        help='Architecture to use [segnet, unet, deconvnet]',
        choices=['segnet', 'unet', 'deconvnet']
    )
    
    parser.add_argument('-d', '--data-path',
        dest='data_path',
        type=str,
        help='Path to the data file in numpy or segy format'
    )
    parser.add_argument('-l', '--labels-path',
        dest='labels_path',
        type=str,
        help='Path to the labels file in numpy format'
    )
    parser.add_argument('-t', '--train',
        dest='train',
        action='store_true',
        default=False,
        help='Whether to train a model or to simply test from a stored model'
    )
    parser.add_argument('-b', '--batch-size',
        dest='batch_size',
        type=int,
        default=16,
        help='Batch Size'
    )
    parser.add_argument('-D', '--device',
        dest='device',
        type=str,
        default='cuda:0',
        help='Device to train on [cuda:n]'
    )
    parser.add_argument('-v', '--cross-validation',
        dest='cross_validation',
        action='store_true',
        default=False,
        help='Whether to use 5-fold cross validation'
    )
    parser.add_argument('-L', '--loss-function',
        dest='loss_function',
        type=str,
        default='cel',
        help='Loss function to use [cel (Cross_Entropy Loss)]',
        choices=['cel']
    )
    parser.add_argument('-o', '--optimizer',
        dest='optimizer',
        type=str,
        default='adam',
        help='Optimizer to use [adam, sgd (Stochastic Gradient Descent)]',
        choices=['adam', 'sgd']
    )
    parser.add_argument('-r', '--learning-rate',
        dest='learning_rate',
        type=float,
        default=1e-4,
        help='Learning rate'
    )
    parser.add_argument('-w', '--weight-decay',
        dest='weight_decay',
        type=float,
        default=1e-5,
        help='L2 regularization. Value 0 indicates no weight decay'
    )
    parser.add_argument('-W', '--weighted-loss',
        dest='weighted_loss',
        action='store_true',
        default=False,
        help='Whether to use class weights in the loss function'
    )
    parser.add_argument('-e', '--n-epochs',
        dest='n_epochs',
        type=int,
        default=20,
        help='Number of epochs'
    )
    parser.add_argument('-O', '--orientation',
        dest='orientation',
        type=str,
        default='in',
        help='Whether the model should be trained using inlines or crosslines',
        choices=['in', 'cross']
    )
    
    parser.add_argument('-f', '--faulty-slices-list',
        dest='faulty_slices_list',
        type=str,
        default=None,
        help='Path to a json file containing a list of faulty slices to remove'
    )
    
    parser.add_argument('-F', '--few-shot',
        dest='few_shot',
        action='store_true',
        default=False,
        help='Whether to swap the train and test sets to train on less data'
    )
    parser.add_argument('-m', '--model-path',
        dest='model_path',
        type=str,
        default=None,
        help='Directory for loading saved model'
    )
    parser.add_argument('-p', '--results-path',
        dest='results_path',
        type=str,
        default=os.path.join(os.getcwd(), 'results'),
        help='Directory for storing execution results'
    )
    
    args = parser.parse_args(args=None)
    run(args)